## Load the paramaters and imports and set some variables

In [1]:
from fuzzywuzzy import fuzz, process
from loguru import logger
import pandas as pd
import sys
import pyodbc

global MSSQL_SERVER
global DATABASE_NAME
MSSQL_SERVER = "STRDVPDBSQL01"
DATABASE_NAME = "masterdata_steam"
conn = pyodbc.connect("DRIVER={SQL Server};SERVER="
                        + MSSQL_SERVER
                        + ";DATABASE="
                        + DATABASE_NAME
                        + ";Trusted_Connection=yes",
                        timeout=15,
                    )
print(f"Connected to SQL Server {MSSQL_SERVER} and table {DATABASE_NAME}")

Connected to SQL Server STRDVPDBSQL01 and table masterdata_steam


### Querry used to get the data

In [3]:
def load_components_from_sql() -> pd.DataFrame:
    cursor = conn.cursor()
    query = ("""
    select 
	comp.[Server]
    ,comp.[internal_article_number]
    ,comp.[article_hash]
    ,comp.[componentname]
    ,comp.[componentidentifier]
    ,comp.[manufacturer]
    ,comp.[manufacturer_description]
    ,comp.[component_erpreferencenumber]
    ,comp.[implant]
    FROM [masterdata_steam].[dbo].[Components] comp
    order by comp.[Server]
    """
    )
    with conn:
        cursor.execute(query)
        query_result = [
            dict(line)
            for line in [
                zip([column[0] for column in cursor.description], row)
                for row in cursor.fetchall()
            ]
        ]
        df = query_result
    return df

### execute the query and store the results in a dataframe

In [4]:
_components_df = pd.DataFrame(load_components_from_sql())
_components_df.to_pickle("components.pkl", compression="gzip")
_components_df.to_csv("all_components.csv", index=False, encoding="utf-8", sep=";")

# _components_df = pd.read_pickle("components.pkl", compression="gzip")
print(f"{len(_components_df)} components loaded")

68824 components loaded


### Check to see if the data is loaded

## Manufacturers
* create dataframe from the grouped manufacturers
* create a csv document with the data

In [ ]:
manufacturers = _components_df.groupby(['manufacturer_description'])['manufacturer_description'].count().sort_values(ascending=False)
manufacturers.explode().to_csv("manufacturer_count.csv", sep=";", index_label=["manufacturer_description", "count"])

In [ ]:
_df_manufacturer_count = manufacturers.sort_values(ascending=False).explode().to_frame()
# print(_df_manufacturer_count)
_manufacturers_with_low_component_count = _df_manufacturer_count.where(_df_manufacturer_count < 10).dropna()
_manufacturers_with_low_component_count.to_csv("manufacturer_with_low_component_count.csv", sep=";", index_label=["manufacturer_description", "count"])
print(_manufacturers_with_low_component_count)

## find similar manufacturers

In [ ]:
from collections import deque
from functools import lru_cache

@lru_cache()
def process_manufacturers():
    _similar_manufacturers = {}
    _count = len(manufacturers)
    # _count = 100
    print(f"{_count} manufacturers to check")
    _ratio: int = 85
    _queue_one = deque(manufacturers.keys())

    # compare all manufacturers with each other and pop them from the queue if they are similar
    for _ in range(_count):
        _queue_len=len(_queue_one)
        if _queue_len == 0:
            break
        _manufacturer_one = _queue_one.popleft()

        for _manufacturer_two in _queue_one:
            _similarity = fuzz.WRatio(_manufacturer_one, _manufacturer_two)
            if _similarity >= _ratio:
                if _manufacturer_one not in _similar_manufacturers:
                    _similar_manufacturers[_manufacturer_one] = []
                    print(f"L1: create {_manufacturer_one}",end="\r")
                if _manufacturer_two not in _similar_manufacturers[_manufacturer_one]:
                    _similar_manufacturers[_manufacturer_one].append(_manufacturer_two)
                    print(f"L2: {_manufacturer_one} and {_manufacturer_two} are similar with {_similarity}% queue len: {_queue_len}",end="\r")
                _queue_one.remove((_manufacturer_two))
                break

    print(" " * 200, end="\r")
    print(f"{len(_similar_manufacturers)} similar manufacturers found with {_ratio}% similarity")

    df_similar_manufacturers = pd.DataFrame.from_dict(_similar_manufacturers, orient='index')
    df_similar_manufacturers.to_csv(f"similar_manufacturers_with{_ratio}_ratio.csv", sep=";")


process_manufacturers()

In [ ]:
from matplotlib import pyplot as plt

_total_count_components = len(_components_df)
_total_count_unique_components = len(_unique_componentidentifiers)
_total_count_componts_with_gr = len(_components_df.query("componentidentifier.str.startswith('GR')"))
_total_count_componts_with_ir = len(_components_df.query("componentidentifier.str.startswith('IR')"))
# _unique_customertokens = _components_df.customertoken.unique()
#loop through the customertokens and count the number of components for each one

# _customertoken_component_count = {}
# for _customertoken in _unique_customertokens:
#     _count_components_for_customertoken = len(_components_df.query("customertoken == @_customertoken"))
#     _customertoken_component_count[_customertoken] = _count_components_for_customertoken

# _customertoken_component_count_df = pd.DataFrame.from_dict(_customertoken_component_count, orient='index', columns=['count'])
# _customertoken_component_count_df.reset_index(inplace=True)
# _customertoken_component_count_df = _customertoken_component_count_df.rename(columns={'index': 'customertoken'})
# _customertoken_component_count_df.dropna(inplace=True)

# print(f"{_total_count_components} total components found")
# print(f"{_total_count_unique_components} total unique components found")
# print(f"{_total_count_componts_with_gr} total components with GR found")
# print(f"{_total_count_componts_with_ir} total components with IR found")

# _customertoken_component_count_series =  _customertoken_component_count_df.query("count > 2")

#plot some data

# _customertoken_component_count_series.plot(kind='pie', x='customertoken', y='count',
#                                         figsize=(10,10), legend=True, autopct='%1.1f%%', fontsize=14, title="Customertoken component count",
#                                         labels=_customertoken_component_count_series.customertoken,
#                                         startangle=90,
#                                         shadow=True)

# plt.legend(loc='upper left', bbox_to_anchor=(1, 1),fontsize=12)

# plt.savefig("output/customertoken_component_count.png")
# plt.show()

_plt_df = pd.DataFrame({"Total Components": [_total_count_components],
                        "Unique Components": [_total_count_unique_components],
                        "Components starting with GR": [_total_count_componts_with_gr],
                        "Components starting with IR": [_total_count_componts_with_ir]})

_plt_df.plot(kind='bar', stacked=False, figsize=(8,6), legend=True, fontsize=14, title='Total Components',
            edgecolor='black', linewidth=1)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=14)
plt.text(-0.21, 0.0, f'{_total_count_components}',fontsize=14, color='black', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.1'))
plt.text(-0.10, 0.0, f'{_total_count_unique_components}', fontsize=14, color='black', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.1'))
plt.text( 0.04, 0.0, f'{_total_count_componts_with_gr}', fontsize=14, color='black', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.1'))
plt.text(0.15, 0.0, f'{_total_count_componts_with_ir}', fontsize=14, color='black', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.1'))
plt.savefig('total_components.png', bbox_inches='tight')
plt.show()

## ComponentIdentifiers

In [5]:
_components_df.query("componentidentifier.str.startswith('GR')").to_csv("components_with_gr.csv", sep=";", encoding='utf-8')
_components_df.query("componentidentifier.str.startswith('IR')").to_csv("components_with_ir.csv", sep=";", encoding='utf-8')

_unique_componentidentifiers = _components_df.groupby(['componentidentifier'])['componentidentifier'].count()
_unique_componentidentifiers_dict = _unique_componentidentifiers.sort_values(ascending=False).explode().to_dict()
_unique_componentidentifiers_df = pd.DataFrame.from_dict(_unique_componentidentifiers_dict, orient='index', columns=['componentidentifier'])
_unique_componentidentifiers_df.to_csv("componentidentifier_count.csv", sep=";", index_label=["componentidentifier", "count"])
print(f"{len(_unique_componentidentifiers)} unique componentidentifiers found")


61442 unique componentidentifiers found


In [6]:
#filter the componentidentifiers
_filterd_component_identifiers = _components_df.copy()

_filterd_component_identifiers = _filterd_component_identifiers[~(_filterd_component_identifiers['componentidentifier'].str.startswith('GR'))
                                & ~(_filterd_component_identifiers['componentidentifier'].str.startswith('IR'))
                                # & ~(_filterd_component_identifiers['componentidentifier'].str.startswith('Unk'))
                                # & ~(_filterd_component_identifiers['componentidentifier'].str.startswith('unknown'))
                                # & ~(_filterd_component_identifiers['componentidentifier'].str.startswith('Unknown'))
                                # & ~(_filterd_component_identifiers['componentidentifier'].str.startswith('unk'))
                                ]
_filterd_component_identifiers.to_csv("components_without_gr_ir.csv", sep=";", index_label=["componentidentifier", "count"])
_filterd_component_identifiers.drop(columns=['manufacturer'], inplace=True)
_filterd_component_identifiers.drop(columns=['manufacturer_description'], inplace=True)
_filterd_component_identifiers.drop(columns=['implant'], inplace=True)
_filterd_component_identifiers.drop(columns=['component_erpreferencenumber'], inplace=True)
print(len(_filterd_component_identifiers))
print(_filterd_component_identifiers.iloc[0])


59337
Server                                                             KPRDCSA01
internal_article_number                                           9025006320
article_hash               6903d9fdf40253dd6708891b4930bea5830dbd31573662...
componentname                EP-FIT/BOFOR/PE/MPF\POLARCUP Reamer 57 [130888]
componentidentifier                                                 75003423
Name: 0, dtype: object


In [ ]:
for col in _filterd_component_identifiers.columns:
    print(f"{col}")


In [7]:
#dict for componentidentifiers
_comp_dict={}
for i, row in _filterd_component_identifiers.iterrows():
    if row['article_hash'] not in _comp_dict:
        _comp_dict[row['article_hash']] = []
        _comp_dict[row['article_hash']].append(row['componentidentifier'])
        _comp_dict[row['article_hash']].append(row['componentname'])
        _comp_dict[row['article_hash']].append(row['Server'])


In [8]:
_lookup_dict = {}
for key, value in _comp_dict.items():
    _lookup_dict[key] = value
print(len(_lookup_dict))


59337


In [9]:
def save_similar_component(master_article_hash: str, article_hash: str, id_linked: bool, name_linked: bool):

    cursor = conn.cursor()
    query = (f"""
            if not exists (
            select master_article_hash
                from dbo.similar_components
            where master_article_hash = '{master_article_hash}'
            and article_hash = '{article_hash}'
            and id_linked = {0 if id_linked == False else 1}
            and name_linked = {0 if name_linked == False else 1}
            )
            insert into dbo.similar_components
            (
            [master_article_hash]
            , [article_hash]
            , [id_linked]
            , [name_linked]
            )
            values
            (
            '{master_article_hash}',
            '{article_hash}',
            { 0 if id_linked == False else 1},
            { 0 if name_linked == False else 1}
            )
            """)
    with conn.cursor() as cursor:
        cursor.execute(query)
        conn.commit()


In [15]:
def cache_previous_processed_article_hashes() -> list:

    _key_list = []
    cursor = conn.cursor()
    query = (f"""
        SELECT distinct master_article_hash from masterdata_steam.dbo.similar_components
            """)
    with conn.cursor() as cursor:
        cursor.execute(query)
        for row in cursor:
            _key_list.append(row[0])

    return _key_list


In [22]:
from fuzzywuzzy import process
from functools import lru_cache
import csv


_check_dict = _lookup_dict.copy()
_comp_id_lookup_dict = _lookup_dict.copy()


_previous_processed_article_hashes = cache_previous_processed_article_hashes()
# remove the previous processed article hashes from the lookup dict
_comp_id_lookup_dict_final = {}

for key, value in _check_dict.items():
    if key not in _previous_processed_article_hashes:
        _comp_id_lookup_dict_final[key] = value

print(f"Number of articles to process: {len(_check_dict)} before removing the previous processed articles")
print(f'Number of articles to process: {len(_comp_id_lookup_dict_final)}, Number of articles already processed: {len(_previous_processed_article_hashes)}')

@lru_cache()
def process_component():

    global _ratio
    _similar_componentidentifiers = {}
    _ratio = 85
    _count = len(_comp_id_lookup_dict_final)
    # _count = 10

    for _ in range(_count):

        _t = _comp_id_lookup_dict_final.popitem()
        _componentidentifier_one= _t[1][0]
        _componentidentifier_one_hash = _t[0]
        _componentname_one = _t[1][1]
        print(f"Processing: {_componentidentifier_one} {_componentname_one} {_componentidentifier_one_hash}"
            f" remaining: {len(_comp_id_lookup_dict_final)}")

        #check if the componentid is in _comp_id_lookup_dict
        for _componentidentifier_two_hash, _t in _comp_id_lookup_dict.items():

            _componentidentifier_two = _t[0]
            _componentname_two = _t[1]


            if _componentidentifier_one == _componentidentifier_two:
                continue
            if _componentname_one == _componentname_two:
                continue

            _compid_similarity = fuzz.ratio(_componentidentifier_one, _componentidentifier_two)
            _compname_similarity = fuzz.token_sort_ratio(_componentname_one, _componentname_two) #Wratio is the weighted ratio

            if _compid_similarity >= _ratio:
                if _componentidentifier_one_hash not in _similar_componentidentifiers:
                    _similar_componentidentifiers[_componentidentifier_one_hash] = []
                    _similar_componentidentifiers[_componentidentifier_one_hash].append((_componentidentifier_one_hash, _componentidentifier_one, _componentname_one))
                    save_similar_component(str(_componentidentifier_one_hash), str(_componentidentifier_two_hash), True, False)


                if _componentidentifier_two not in _similar_componentidentifiers[_componentidentifier_one_hash]:
                    _similar_componentidentifiers[_componentidentifier_one_hash].append((_componentidentifier_two, _componentidentifier_two_hash))
                    save_similar_component(str(_componentidentifier_one_hash), str(_componentidentifier_two_hash), True, False)

                if _compname_similarity >= _ratio:

                    if _componentname_two not in _similar_componentidentifiers[_componentidentifier_one_hash]:
                        _similar_componentidentifiers[_componentidentifier_one_hash].append((_componentname_two, _componentidentifier_two_hash))
                        save_similar_component(str(_componentidentifier_one_hash), str(_componentidentifier_two_hash), False, True)
        _count -= 1
        del _comp_id_lookup_dict[_componentidentifier_two_hash]

process_component()

Number of articles to process: 59337 before removing the previous processed articles
Number of articles to process: 57587, Number of articles already processed: 1823
Processing: 28136 BT obturateur avec pointe L:200mm 142e8ebbccdb9d4651bb5386949ff3f239c1440d1c726062258c701e12775601 remaining: 57586
Processing: 7111-2743 barre de fixation L:203mm 7355b3d3e44732e4d86702b3b503d19d49e70060a7e3dc35a8d50361e7b6f9a7 remaining: 57585
Processing: 24.614.32 crochet L:100mm 846f43fd3b6cdfb8413d50a8499d2c4d4ac705fe9cdb4614bb876a46d135b400 remaining: 57584
Processing: 26046 AA optique 0° 5mm L:360mm 96a97503b06861f9f3c5352c3a072cb5a0bbc95786aa9a7e9413f3ff0686da9b remaining: 57583
Processing: 11.0031a Optique 0° L:410mm 37dd5e3a0475b97c04e6f3151e121f1da30d607783741ef5edf0d352dd9659f0 remaining: 57582
Processing: IM220-020 petit embout pour câble lumière froide L:20mm d8977d4ea52fc62bd9003bd4fa8e1d5cfe8691b0a6fe0b630c557dd74f381306 remaining: 57581
Processing: IM202-231 câble lumière froide 15da26d82

Error: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionWrite (send()). (10054) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')